In [1]:
import dgl
import torch
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.tri import Triangulation
from IPython.display import display, clear_output
import os
import wandb
import copy

Using backend: pytorch


In [2]:
def triplot(coordinates,elem,values,figure_size):
    tri_obj = Triangulation(coordinates[:,0],coordinates[:,1],elem-1)
    fig = plt.figure(figsize=figure_size)
    ax=fig.add_subplot(1,1,1)
    ax.triplot(tri_obj,c='black')
    ax.tricontourf(tri_obj,values.reshape(-1),levels=np.linspace(-0.01,1.01,51),cmap='Greys')
    plt.xticks([])
    plt.yticks([])
    plt.gcf().set_facecolor("white")
    
def collate(samples):
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    num_nodes = batched_graph.batch_num_nodes()
    return batched_graph, torch.tensor(np.vstack(labels))

def evaluate(model, loader):
    model.eval()
    with torch.no_grad():
        for disjoint_graphs_test, output_features_test in loader:
            predictions_test = model(disjoint_graphs_test.to('cuda'), disjoint_graphs_test.ndata['Features'].to('cuda'))
            loss_eval = torch.nn.L1Loss()(output_features_test.to('cuda'), predictions_test)
            loss_eval = loss_eval.to('cpu').detach().numpy()
            break
    return loss_eval

def plot_save(model,epoch,path):
    data_amount = [141,280,153,125,252,125,141,280,153]
    foldermat = ['C_C','C_N','C_R','L_C','L_N','L_R','M_C','M_N','M_R']
    
    os.makedirs(path + '/' + '%d_epoch'%(epoch), exist_ok=True)
    
    for folder_name in foldermat:
        folder_ind = foldermat.index(folder_name)
        num_data = data_amount[folder_ind]
        with open(folder_name + '/' + folder_name + '_test_index.bin','rb') as test_ind:
            plot_test_ind = np.fromfile(test_ind,dtype = 'float32',count=-1).astype(int)
            plot_test_ind = plot_test_ind-1
        with open(folder_name + '/' + 'edgeinfo.bin','rb') as elen:
            edge_num_mat = np.fromfile(elen,dtype = 'float32',count=-1).astype(int)
        with open(folder_name + '/infomat.bin','rb') as im:
            infomat = np.fromfile(im,dtype = 'float32',count=-1).reshape(num_data,13).transpose([1,0])
        with open(folder_name + '/' + 'ELEMinfo.bin','rb') as elem:
            elemmat = np.fromfile(elem,dtype = 'float32',count=-1).astype(int)
            
        randind = np.random.choice(plot_test_ind, replace=False)
        node_num = int(infomat[0,randind])
        
        with open(folder_name + '/' + 'A_features_%d.bin'%(randind+1),'rb') as af_plot:
            afmat_plot = np.fromfile(af_plot,dtype='float32',count=-1).reshape(node_num+9,node_num).transpose([1,0])
            afmat_plot[:,-1:] = np.clip(afmat_plot[:,-1:],0,1)

        with open(folder_name + '/' + 'edges_%d.bin'%(randind+1),'rb') as em_plot:
            edge_mat_plot = np.fromfile(em_plot,dtype='float32',count=-1).reshape(2,edge_num_mat[randind]).transpose([1,0])
            edge_mat_plot = edge_mat_plot.astype(int)
            edge_mat_plot += -1
            
        with open(folder_name + '/' + 'ELEM_%d.bin'%(randind+1),'rb') as elem_plot:
            ELEM_plot = np.fromfile(elem_plot,dtype='float32',count=-1).reshape(3,elemmat[randind]).transpose([1,0]).astype('int')
        
        graph_plot = dgl.graph((edge_mat_plot[:,0],edge_mat_plot[:,1]))
        graph_plot.ndata['Features'] = torch.tensor(afmat_plot[:,-9:-1])
        graph_plot = dgl.add_self_loop(graph_plot)
        graph_plot = dgl.to_bidirected(graph_plot,copy_ndata = True)
        
        model = model.to('cpu')
        model.eval()
        with torch.no_grad():
            predictions_plot = model(graph_plot,graph_plot.ndata['Features'])
        
        plot_hight = np.max(afmat_plot[:,-4:-3])-np.min(afmat_plot[:,-4:-3])
        plot_width = np.max(afmat_plot[:,-3:-2])-np.min(afmat_plot[:,-3:-2])
        
        triplot(afmat_plot[:,-4:-2],ELEM_plot,afmat_plot[:,-1:],(plot_hight,plot_width))
        plt.savefig(path + '/' + '%d_epoch'%(epoch) + '/' + folder_name+'_TO.jpg')
        plt.close()
        triplot(afmat_plot[:,-4:-2],ELEM_plot,predictions_plot,(plot_hight,plot_width))
        plt.savefig(path + '/' + '%d_epoch'%(epoch) + '/' + folder_name+'_DL.jpg')
        plt.close()
        
    model = model.to('cuda')

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def reg_layer(layer_num):
    assert len(np.array(layer_num).shape) == 1
    num_reg_layer = len(layer_num)
    namespace = []
    for ln in range(len(layer_num)):
        namespace.append('graph_conv%d'%(layer_num[ln])+'.weight')
    return namespace

In [3]:
class Mesh2Graph(dgl.data.DGLDataset):
    def __init__(self,is_train):
        self.is_train = is_train
        super().__init__(name='MESH2GRAPH'+ '__' + is_train)
        
    def process(self):
        self.graphs = []
        self.labels = []
        
        path = os.getcwd()
        folder_list = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name)) and not name.startswith('.')]
        folder_list_sort = ['C_C','C_N','C_R','L_C','L_N','L_R','M_C','M_N','M_R']
        data_amount = [141,280,153,125,252,125,141,280,153]

        for folder_iter in range(len(folder_list)):
            folder_name = folder_list[folder_iter]
            ind_folder = folder_list_sort.index(folder_name)
            num_data = data_amount[ind_folder]

            with open(folder_name + '/' + 'edgeinfo.bin','rb') as elen:
                edge_num = np.fromfile(elen,dtype = 'float32',count=-1).astype(int)

            with open(folder_name + '/' + folder_name + '_test_index.bin','rb') as pind:
                part_test_ind = np.fromfile(pind,dtype = 'float32',count=-1).astype(int)
                part_test_ind = part_test_ind-1

            with open(folder_name + '/infomat.bin','rb') as im:
                infomat = np.fromfile(im,dtype = 'float32',count=-1).reshape(num_data,13).transpose([1,0])

            for data_iter in range(num_data):
                node_num = int(infomat[0,data_iter])

                with open(folder_list[folder_iter] + '/' + 'A_features_%d.bin'%(data_iter+1),'rb') as af:
                    afmat = np.fromfile(af,dtype='float32',count=-1).reshape(node_num+9,node_num).transpose([1,0])
                    afmat[:,-1:] = np.clip(afmat[:,-1:],0,1)

                with open(folder_list[folder_iter] + '/' + 'edges_%d.bin'%(data_iter+1),'rb') as em:
                    edge_mat = np.fromfile(em,dtype='float32',count=-1).reshape(2,edge_num[data_iter]).transpose([1,0])
                    edge_mat = edge_mat.astype(int)
                    edge_mat += -1
                    
                node_features = afmat[0:,node_num:-1]
                node_label = afmat[0:,-1:]
                
                g = dgl.graph((edge_mat[:,0],edge_mat[:,1]))
                g.ndata['Features'] = torch.tensor(node_features)
                g = dgl.add_self_loop(g)
                g = dgl.to_bidirected(g,copy_ndata = True)

                clear_output(wait=True)
                if (self.is_train is 'train') and (data_iter not in part_test_ind):
                    self.graphs.append(g)
                    self.labels.append(node_label)
                elif (self.is_train is 'test') and (data_iter in part_test_ind):
                    self.graphs.append(g)
                    self.labels.append(node_label)
                elif (self.is_train is 'total'):
                    self.graphs.append(g)
                    self.labels.append(node_label)
                        
                print(folder_name, g, g.device)
                clear_output(wait=True)
                    
        print('Graph Representation Done')
    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]
    
    def __len__(self):
        return len(self.graphs)

In [4]:
class TOP_MODEL(torch.nn.Module):
    def __init__(self, num_layers, num_hidden_features):
        super().__init__()
        assert num_layers%2 ==0, 'Number of Layers is not Even : Skip connection operation is obscure'
        assert num_hidden_features < 512, 'Number of Filter is too Large'
        
        self.num_layers = num_layers
        self.num_hidden_features = num_hidden_features

        for nl in range(1,self.num_layers+1):
            if nl==1:
                self.graph_conv1 = dgl.nn.pytorch.conv.GraphConv(in_feats=8,out_feats=self.num_hidden_features,bias=False)
                self.bn1 = torch.nn.BatchNorm1d(num_features = self.num_hidden_features)
                self.act1 = torch.nn.ReLU()
            
            elif nl in [8,16,17,25,32]:
                in_GCN_tmp = "self.graph_conv%d"%(nl)
                in_BN_tmp = "self.bn%d"%(nl)
                in_ACT_tmp = "self.act%d"%(nl)
                
                GCN_tmp = "dgl.nn.pytorch.conv.GraphConv(in_feats=self.num_hidden_features,out_feats=self.num_hidden_features,bias=False)"
                BN_tmp = "torch.nn.BatchNorm1d(num_features = self.num_hidden_features)"
                ACT_tmp = "torch.nn.ReLU()"
                
                exec('%s = %s'%(in_GCN_tmp,GCN_tmp))
                exec('%s = %s'%(in_BN_tmp,BN_tmp))
                exec('%s = %s'%(in_ACT_tmp,ACT_tmp))
            elif ((nl<=16) and (nl%2==0)) or ((nl>=17) and (nl%2==1)):
                in_GCN_tmp = "self.graph_conv%d"%(nl)
                in_DO_tmp = "self.do%d"%(nl)
                in_ACT_tmp = "self.act%d"%(nl)

                GCN_tmp = "dgl.nn.pytorch.conv.GraphConv(in_feats=self.num_hidden_features,out_feats=self.num_hidden_features,bias=True)"
                DO_tmp = "torch.nn.Dropout(p=0.2)"
                ACT_tmp = "torch.nn.ReLU()"

                exec('%s = %s'%(in_GCN_tmp,GCN_tmp))
                exec('%s = %s'%(in_DO_tmp,DO_tmp))
                exec('%s = %s'%(in_ACT_tmp,ACT_tmp))
            elif ((nl<=16) and (nl%2==1)) or ((nl>=17) and (nl%2==0)):
                in_GCN_tmp = "self.graph_conv%d"%(nl)
                in_ACT_tmp = "self.act%d"%(nl)

                GCN_tmp = "dgl.nn.pytorch.conv.GraphConv(in_feats=self.num_hidden_features,out_feats=self.num_hidden_features,bias=True)"
                ACT_tmp = "torch.nn.ReLU()"

                exec('%s = %s'%(in_GCN_tmp,GCN_tmp))
                exec('%s = %s'%(in_ACT_tmp,ACT_tmp))

        self.graph_conv_last = dgl.nn.pytorch.conv.GraphConv(in_feats=self.num_hidden_features,out_feats=1,bias=True)
        self.act_last = torch.nn.Sigmoid()
        
    def forward(self, graphs, features):
        for i in range(1,self.num_layers+1):
            if i==1:
                self.c1 = self.graph_conv1(graphs, features)
                self.b1 = self.bn1(self.c1)            
                self.a1 = self.act1(self.b1)
            elif i in [8,16]:
                exec('self.c%d = self.graph_conv%d(graphs, self.a%d)'%(i,i,i-1))
                exec('self.b%d = self.bn%d(self.c%d)'%(i,i,i))
                exec('self.a%d = self.act%d(self.b%d)'%(i,i,i))
            elif (i%2==0) and (i<=16) and (i not in [8,16]):
                exec('self.c%d = self.graph_conv%d(graphs, self.a%d)'%(i,i,i-1))
                exec('self.d%d = self.do%d(self.c%d)'%(i,i,i))
                exec('self.a%d = self.act%d(self.d%d)'%(i,i,i))
            elif (i%2==1) and (i<=16) and (i not in [8,16]):
                exec('self.c%d = self.graph_conv%d(graphs, self.a%d)'%(i,i,i-1))
                exec('self.a%d = self.act%d(self.c%d)'%(i,i,i))
            elif i in [17,25,32]:
                exec('self.c%d = self.graph_conv%d(graphs, self.a%d)'%(i,i,i-1))
                exec('self.b%d = self.bn%d(self.c%d)'%(i,i,i))
                exec('self.skip%d = self.a%d + self.b%d'%(i,self.num_layers+1-i,i))
                exec('self.a%d = self.act%d(self.skip%d)'%(i,i,i))
            elif (i%2==0) and (i>=17) and (i not in [17,25,32]):
                exec('self.c%d = self.graph_conv%d(graphs, self.a%d)'%(i,i,i-1))
                exec('self.skip%d = self.a%d + self.c%d'%(i,self.num_layers+1-i,i))
                exec('self.a%d = self.act%d(self.skip%d)'%(i,i,i))
            elif (i%2==1) and (i>=17) and (i not in [17,25,32]):
                exec('self.c%d = self.graph_conv%d(graphs, self.a%d)'%(i,i,i-1))
                exec('self.d%d = self.do%d(self.c%d)'%(i,i,i))
                exec('self.skip%d = self.a%d + self.d%d'%(i,self.num_layers+1-i,i))
                exec('self.a%d = self.act%d(self.skip%d)'%(i,i,i))
            
        exec('self.c_last = self.graph_conv_last(graphs, self.a%d)'%(self.num_layers))
        out = self.act_last(self.c_last)
        
        return out

In [5]:
train_dataset = Mesh2Graph('train')
test_dataset =  Mesh2Graph('test')

Graph Representation Done


In [6]:
num_layers = 32
num_hidden_features = 128
model = TOP_MODEL(num_layers,num_hidden_features)
model = model.to('cuda')

In [7]:
train_loader = dgl.dataloading.pytorch.GraphDataLoader(train_dataset, collate_fn=collate, batch_size=128, shuffle=True)
test_loader = dgl.dataloading.pytorch.GraphDataLoader(test_dataset, collate_fn=collate, batch_size=128, shuffle=True)

In [8]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,factor=0.9, patience=50, min_lr=1e-5, verbose=False)
MAE_loss = torch.nn.L1Loss()

In [9]:
reg_layer_num = []
for ln in range(1,num_layers+1):
    if (ln%2==1) and (ln<=16) and (ln not in [1,8,16]):
        reg_layer_num.append(ln)
    elif (ln%2==0) and (ln>=17) and (ln not in [17,25,32]):
        reg_layer_num.append(ln)
reg_layer_namespace = reg_layer(reg_layer_num)

In [10]:
emax = 2000
min_test_err = [1]
modelsave_interval = 100
plotsave_interval = 100
l2reg_factor = 0.01
experiment_name = "GNN_comsol_L2reg_128bs"
wandb.init(project="GNN_bcs2opt",group=experiment_name)
config = wandb.config

save_fig_path = '../Result_pictures/L2reg_128bs'
save_interv_model_path = '../models/initial_try/L2reg_128bs'
save_best_model_path = '../models/initial_try/best_save_L2reg_128bs'

os.makedirs(save_fig_path, exist_ok=True)
os.makedirs(save_interv_model_path, exist_ok=True)
os.makedirs(save_best_model_path, exist_ok=True)

for e in range(1,emax+1):
    for batch_num, (disjoint_graphs, output_features) in enumerate(train_loader):
        pred_features = model(disjoint_graphs.to('cuda'), disjoint_graphs.ndata['Features'].to('cuda'))
        loss_training = MAE_loss(pred_features, output_features.to('cuda'))
        loss_reg = 0
        for parameter in model.named_parameters():
            for i in range(len(reg_layer_namespace)):
                if reg_layer_namespace[i] in parameter:
                    loss_reg += torch.sum(parameter[1]**2)
        loss_training = loss_training + l2reg_factor*loss_reg
        optimizer.zero_grad()
        loss_training.backward()
        optimizer.step()
        print('Epoch : %d     Batch : %d     Training loss : %f     reg loss : %f     Learning rate : %f'%(e,batch_num,loss_training,loss_reg,get_lr(optimizer)))
        clear_output(wait = True)
    loss_test = evaluate(model,test_loader)
    scheduler.step(loss_test)
    wandb.log({"training_loss":loss_training, "test_loss":loss_test})
    
    if e%plotsave_interval ==0:
        plot_save(model,e,save_fig_path)   
    if e%modelsave_interval == 0:
        torch.save(model.state_dict(),save_interv_model_path + '/' + 'model_%d_epoch'%(e))
    if min_test_err > loss_test:
        torch.save(model.state_dict(),save_best_model_path + '/' + 'best_model')
        min_test_err = copy.copy([loss_test])

In [ ]:
model_load = TOP_MODEL(32,128)
model_load.load_state_dict(torch.load(savemodel_path + 'model_test'))

In [ ]:
plot_save(model_load,1,'test')

In [ ]:
for i in range(1,32):
    if ((i<=16) and (i%2==1)) or ((i>=17) and (i%2==0)):
        print(i)

In [ ]:
for disjoint_graphs, output_features in train_loader:
    test_output = model(disjoint_graphs, disjoint_graphs.ndata['Features'])
    print(test_output, output_features)
    break

In [ ]:
for a in dir(train_dataset):
    print(a)

In [ ]:
print(g.edges(form = 'all'))

In [ ]:
g.ndata['x'] = torch.ones(g.num_nodes(),4)
g.ndata['y'] = torch.ones(g.num_nodes(),4)
g

In [ ]:
dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

In [ ]:
g = dataset[0]
print(g.ndata)